<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Les-imports-et-configuration-du-document" data-toc-modified-id="Les-imports-et-configuration-du-document-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Les imports et configuration du document</a></span><ul class="toc-item"><li><span><a href="#La-lecture-des-fichiers-d'un-répértoire" data-toc-modified-id="La-lecture-des-fichiers-d'un-répértoire-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>La lecture des fichiers d'un répértoire</a></span></li></ul></li><li><span><a href="#La-temperature-mensuelle" data-toc-modified-id="La-temperature-mensuelle-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>La temperature mensuelle</a></span></li><li><span><a href="#La-météo-mensuelle" data-toc-modified-id="La-météo-mensuelle-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>La météo mensuelle</a></span></li><li><span><a href="#La-météo-----hebdomadaire" data-toc-modified-id="La-météo-----hebdomadaire-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>La météo     hebdomadaire</a></span></li><li><span><a href="#La-météo-----journalière" data-toc-modified-id="La-météo-----journalière-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>La météo     journalière</a></span></li></ul></div>

# Les imports et configuration du document

In [1]:
import pandas as pd 
import seaborn as sns
import warnings 
import os
from datetime import datetime
warnings.filterwarnings(action="ignore")

%pylab inline

plt.style.use('seaborn-darkgrid')
sns.set(font_scale=3)

os.chdir(r"C:\Formation\donnees")

Populating the interactive namespace from numpy and matplotlib


In [2]:
!dir

 Le volume dans le lecteur C s'appelle Windows
 Le num‚ro de s‚rie du volume est 56E1-170F

 R‚pertoire de C:\Formation\donnees

08/12/2021  12:08    <DIR>          .
08/12/2021  12:08    <DIR>          ..
29/01/2018  11:42             2ÿ436 50_Startups.csv
26/02/2018  05:29           240ÿ949 abalone.csv
21/02/2018  01:44         1ÿ171ÿ439 agents.json
20/11/2016  12:12           785ÿ065 AirQualityUCI01.csv
03/09/2015  14:39           785ÿ065 AirQualityUCI02.csv
20/11/2016  12:14           785ÿ065 AirQualityUCI03.csv
20/11/2016  12:34           783ÿ119 AirQualityUCI04.csv
22/07/2018  08:35            49ÿ082 boston.housing.data.txt
26/02/2018  05:30           129ÿ652 breastCancer.csv
10/05/2018  14:56         1ÿ423ÿ529 California Housing Price.csv
26/02/2018  14:33       152ÿ996ÿ104 creditCard.csv
25/04/2019  08:16           315ÿ987 creditDesc.csv
07/10/2021  14:37         1ÿ010ÿ610 Data_Cortex_Nuclear.csv
16/09/2021  14:04            23ÿ873 diabetes.csv
13/03/2017  04:25        76ÿ775ÿ0

## La lecture des fichiers d'un répértoire 

In [3]:
def dateparse(d):
    return pd.datetime.strptime(d,'%Y%m%d%H%M%S')

meteo = pd.DataFrame()

for dirname, _, filenames in os.walk('meteo30'):
    for filename in filenames:
        if filename.rfind('synop') >= 0 :
            meteo = pd.concat([meteo,
                               pd.read_csv(os.path.join(dirname, filename),
                                           sep=';',
                                           usecols=['numer_sta','date','dd','ff','t','u','vv',
                                                    'pres','rr1','rr3','rr6','rr12','rr24'],
                                           na_values='mq',
                                           dtype={'numer_sta':str,'date':str},
                                           parse_dates={'DateHeure': ['date']},
                                           date_parser=dateparse
                                          ).rename( columns={'numer_sta':'Station',
                                                              'date':'Date',
                                                              'dd':'DirectionVent',
                                                              'ff':'VitesseVent',
                                                              't':'Temperature',
                                                              'u':'Humidite',
                                                              'vv':'Visibilite',
                                                              'pres':'Pression',
                                                              'rr1' :'Precipitation01',
                                                              'rr3' :'Precipitation03',
                                                              'rr6' :'Precipitation06',
                                                              'rr12':'Precipitation12',
                                                              'rr24':'Precipitation24'})],axis=0)
            
meteo.Temperature  = meteo.Temperature - 273.15
meteo.Pression     = meteo.Pression / 100
meteo.Visibilite   = meteo.Visibilite / 1000
meteo['Mois']      = meteo.DateHeure.dt.month
meteo['Annee']     = meteo.DateHeure.dt.year
meteo['AnneeMois'] = meteo.DateHeure.dt.year*100 + meteo.DateHeure.dt.month
meteo['Semaine']   = meteo.DateHeure.dt.weekofyear
meteo['MoisJour']  = meteo.DateHeure.dt.month*100 + meteo.DateHeure.dt.day
meteo['JourA']     = meteo.DateHeure.dt.dayofyear
meteo.head(5)            

,DateHeure,Station,DirectionVent,VitesseVent,Temperature,Humidite,Visibilite,Pression,Precipitation01,Precipitation03,Precipitation06,Precipitation12,Precipitation24,Mois,Annee,AnneeMois,Semaine,MoisJour,JourA
0,1997-01-01,07005,80.0,6.1,-10.61,78.0,NaN,1009.1,NaN,NaN,0.0,NaN,NaN,1,1997,199701,1,101,1
1,1997-01-01,07015,40.0,5.1,-11.50,77.0,3.0,1013.1,NaN,0.0,0.0,NaN,NaN,1,1997,199701,1,101,1
2,1997-01-01,07020,80.0,15.9,0.80,68.0,NaN,1014.5,NaN,0.0,0.0,NaN,NaN,1,1997,199701,1,101,1
3,1997-01-01,07027,60.0,5.1,-7.90,79.0,6.0,1008.1,NaN,0.0,0.0,NaN,NaN,1,1997,199701,1,101,1
4,1997-01-01,07037,60.0,6.1,-10.20,84.0,NaN,997.2,NaN,NaN,0.0,NaN,NaN,1,1997,199701,1,101,1


In [4]:
meteo.shape

(4144768, 19)

In [5]:
postes = pd.read_csv('postesSynop.csv',sep=';',dtype={'ID':str})
postes.Nom =postes.Nom.apply(lambda x : x if x in ['CLERMONT-FD','MONT-DE-MARSAN',
                                       'ST-PIERRE','ST-BARTHELEMY METEO'] 
                               else x[0:x.find('-')] 
                                    if x.find('-') != -1 else x).apply(lambda x : str(x).title())
postes.Altitude = postes.Altitude.astype('int16')
postes = postes[postes.ID < '08000']

In [6]:
postes.loc[postes.Latitude  < postes.Latitude.mean(),'Zone'] = 'S'
postes.loc[postes.Latitude  > postes.Latitude.mean(),'Zone'] = 'N'
postes.loc[postes.Longitude < postes.Longitude.mean(),'Zone'] += 'O'
postes.loc[postes.Longitude > postes.Longitude.mean(),'Zone'] += 'E'

In [7]:
postes.head()

,ID,Nom,Latitude,Longitude,Altitude,Zone
0,07005,Abbeville,50.136000,1.834000,69,NO
1,07015,Lille,50.570000,3.097500,47,NE
2,07020,Pte De La Hague,49.725167,-1.939833,6,NO
3,07027,Caen,49.180000,-0.456167,67,NO
4,07037,Rouen,49.383000,1.181667,151,NO


In [8]:
meteo = postes.merge(meteo, how = "inner", 
                     left_on = "ID", right_on = "Station").drop(["ID","Station"], axis = "columns")

In [9]:
meteo['Precipitation'] =  meteo['Precipitation03'].combine_first(meteo['Precipitation06']/2)\
                                                  .combine_first(meteo['Precipitation12']/4)\
                                                  .combine_first(meteo['Precipitation24']/8)\
                                                  .combine_first(meteo['Precipitation01']*3)

In [10]:
meteo.drop(columns=['Precipitation06',
                    'Precipitation12',
                    'Precipitation24',
                    'Precipitation01',
                    'Precipitation03'], inplace=True)

In [11]:
meteo.to_parquet('meteo.gzip',compression='gzip', engine='pyarrow')

In [12]:
%%time
meteo = pd.read_parquet('meteo.gzip', engine='pyarrow')
meteo.dtypes

Wall time: 1.19 s


Nom                      object
Latitude                float64
Longitude               float64
Altitude                  int16
Zone                     object
DateHeure        datetime64[ns]
DirectionVent           float64
VitesseVent             float64
Temperature             float64
Humidite                float64
Visibilite              float64
Pression                float64
Mois                      int64
Annee                     int64
AnneeMois                 int64
Semaine                   int64
MoisJour                  int64
JourA                     int64
Precipitation           float64
dtype: object

In [13]:
meteo.to_csv('meteo.csv')

In [14]:
%%time
meteo = pd.read_parquet('meteo.gzip', engine='pyarrow')
meteo.dtypes

Wall time: 1.04 s


Nom                      object
Latitude                float64
Longitude               float64
Altitude                  int16
Zone                     object
DateHeure        datetime64[ns]
DirectionVent           float64
VitesseVent             float64
Temperature             float64
Humidite                float64
Visibilite              float64
Pression                float64
Mois                      int64
Annee                     int64
AnneeMois                 int64
Semaine                   int64
MoisJour                  int64
JourA                     int64
Precipitation           float64
dtype: object

In [15]:
%%time
pd.read_csv('meteo.csv').dtypes

Wall time: 5.34 s


Unnamed: 0         int64
Nom               object
Latitude         float64
Longitude        float64
Altitude           int64
Zone              object
DateHeure         object
DirectionVent    float64
VitesseVent      float64
Temperature      float64
Humidite         float64
Visibilite       float64
Pression         float64
Mois               int64
Annee              int64
AnneeMois          int64
Semaine            int64
MoisJour           int64
JourA              int64
Precipitation    float64
dtype: object

In [16]:
meteo.isna().sum()

Nom                   0
Latitude              0
Longitude             0
Altitude              0
Zone                  0
DateHeure             0
DirectionVent      7331
VitesseVent        6901
Temperature        7938
Humidite          10314
Visibilite       402465
Pression          13419
Mois                  0
Annee                 0
AnneeMois             0
Semaine               0
MoisJour              0
JourA                 0
Precipitation     13940
dtype: int64

In [17]:
meteo.head()

,Nom,Latitude,Longitude,Altitude,Zone,DateHeure,DirectionVent,VitesseVent,Temperature,Humidite,Visibilite,Pression,Mois,Annee,AnneeMois,Semaine,MoisJour,JourA,Precipitation
0,Abbeville,50.136,1.834,69,NO,1997-01-01 00:00:00,80.0,6.1,-10.61,78.0,NaN,1009.1,1,1997,199701,1,101,1,0.0
1,Abbeville,50.136,1.834,69,NO,1997-01-01 03:00:00,60.0,7.2,-11.50,79.0,NaN,1008.8,1,1997,199701,1,101,1,NaN
2,Abbeville,50.136,1.834,69,NO,1997-01-01 06:00:00,70.0,7.2,-11.70,79.0,6.0,1009.2,1,1997,199701,1,101,1,0.0
3,Abbeville,50.136,1.834,69,NO,1997-01-01 09:00:00,80.0,8.2,-11.80,77.0,3.0,1010.1,1,1997,199701,1,101,1,NaN
4,Abbeville,50.136,1.834,69,NO,1997-01-01 12:00:00,70.0,8.2,-10.30,72.0,3.0,1010.4,1,1997,199701,1,101,1,0.0


In [18]:
meteo.columns

Index(['Nom', 'Latitude', 'Longitude', 'Altitude', 'Zone', 'DateHeure',
       'DirectionVent', 'VitesseVent', 'Temperature', 'Humidite', 'Visibilite',
       'Pression', 'Mois', 'Annee', 'AnneeMois', 'Semaine', 'MoisJour',
       'JourA', 'Precipitation'],
      dtype='object')

# La temperature mensuelle

In [20]:
temperatures = meteo[['Nom', 'Latitude', 'Longitude', 'Altitude','Zone', 
                'Temperature','Mois']].pivot_table(index=['Nom', 'Latitude', 'Longitude', 'Altitude','Zone'],
                           columns='Mois')
temperatures.columns = ['janv.','févr.','mars','avr.','mai','juin','juill.','août','sept.','oct.','nov.','déc.']
temperatures.head()

,,,,,janv.,févr.,mars,avr.,mai,juin,juill.,août,sept.,oct.,nov.,déc.
Nom,Latitude,Longitude,Altitude,Zone,,,,,,,,,,,,
Abbeville,50.136000,1.834000,69,NO,4.339488,4.961508,7.159573,9.818468,12.974397,15.831397,17.636887,17.744688,15.344691,11.884938,7.807354,5.241632
Ajaccio,41.918000,8.792667,5,SE,8.845452,9.020222,10.923023,13.630346,17.333195,21.449760,23.778255,23.849876,20.827605,17.428642,13.107521,9.916328
Alencon,48.445500,0.110167,143,NO,4.360548,4.935061,7.315260,10.000312,13.440577,16.795848,18.551608,18.346818,15.402646,11.760500,7.611062,5.067731
Bale,47.614333,7.510000,263,NE,1.997963,3.016738,6.601501,10.567396,14.848118,18.719350,20.052836,19.562820,15.413585,10.943476,5.874690,2.970391
Bastia,42.540667,9.485167,10,SE,9.011034,9.158366,11.222681,13.814283,17.732465,22.010528,24.744154,24.773452,21.194754,17.341999,13.156176,10.097650


In [21]:
temperatures.to_parquet('temperatures.gzip',compression='gzip', engine='pyarrow')

In [22]:
temperatures.to_csv('temperatures.csv')

In [23]:
%%time
temperatures = pd.read_parquet('temperatures.gzip', engine='pyarrow')
temperatures.dtypes

Wall time: 6.98 ms


janv.     float64
févr.     float64
mars      float64
avr.      float64
mai       float64
juin      float64
juill.    float64
août      float64
sept.     float64
oct.      float64
nov.      float64
déc.      float64
dtype: object

In [24]:
temperatures.head()

,,,,,janv.,févr.,mars,avr.,mai,juin,juill.,août,sept.,oct.,nov.,déc.
Nom,Latitude,Longitude,Altitude,Zone,,,,,,,,,,,,
Abbeville,50.136000,1.834000,69,NO,4.339488,4.961508,7.159573,9.818468,12.974397,15.831397,17.636887,17.744688,15.344691,11.884938,7.807354,5.241632
Ajaccio,41.918000,8.792667,5,SE,8.845452,9.020222,10.923023,13.630346,17.333195,21.449760,23.778255,23.849876,20.827605,17.428642,13.107521,9.916328
Alencon,48.445500,0.110167,143,NO,4.360548,4.935061,7.315260,10.000312,13.440577,16.795848,18.551608,18.346818,15.402646,11.760500,7.611062,5.067731
Bale,47.614333,7.510000,263,NE,1.997963,3.016738,6.601501,10.567396,14.848118,18.719350,20.052836,19.562820,15.413585,10.943476,5.874690,2.970391
Bastia,42.540667,9.485167,10,SE,9.011034,9.158366,11.222681,13.814283,17.732465,22.010528,24.744154,24.773452,21.194754,17.341999,13.156176,10.097650


# La météo mensuelle

In [25]:
meteoM = meteo[['Nom', 'Latitude', 'Longitude', 'Altitude','Zone', 
                'VitesseVent', 'Temperature', 'Humidite', 
                'Visibilite','Pression','Precipitation', 
                'Mois']].pivot_table(index=['Nom', 'Latitude', 'Longitude', 'Altitude','Zone'],
                           columns='Mois')
meteoM.columns = [f'{x[1]:02d}{x[0]}' for x in meteoM.columns]
meteoM.head()

,,,,,01Humidite,02Humidite,03Humidite,04Humidite,05Humidite,06Humidite,07Humidite,08Humidite,09Humidite,10Humidite,...,03VitesseVent,04VitesseVent,05VitesseVent,06VitesseVent,07VitesseVent,08VitesseVent,09VitesseVent,10VitesseVent,11VitesseVent,12VitesseVent
Nom,Latitude,Longitude,Altitude,Zone,,,,,,,,,,,,,,,,,,,,,
Abbeville,50.136000,1.834000,69,NO,86.724713,83.442240,81.179400,75.727027,78.056258,79.027077,77.900293,78.544567,80.426669,84.614636,...,4.825237,4.297918,4.199157,3.891925,3.913016,3.722128,3.732194,4.218847,4.305830,4.752122
Ajaccio,41.918000,8.792667,5,SE,77.577222,76.085307,74.541455,74.441117,73.218376,70.520309,67.527176,67.632660,70.216669,74.434473,...,3.278142,3.291029,3.310744,3.441544,3.601828,3.502506,3.381423,3.091826,3.145965,3.233035
Alencon,48.445500,0.110167,143,NO,88.713347,84.135055,79.976677,74.493544,75.708525,74.197207,71.743615,73.341054,76.939384,84.826907,...,3.676683,3.435019,3.412068,3.196204,3.258630,2.959461,2.865299,3.087170,3.266048,3.620345
Bale,47.614333,7.510000,263,NE,84.468035,79.617506,73.838076,69.405712,71.928629,71.442922,71.057795,74.507434,79.220094,83.978486,...,2.889943,2.704043,2.699644,2.451203,2.394586,2.131648,2.148799,2.313112,2.467429,2.889509
Bastia,42.540667,9.485167,10,SE,73.027388,71.772398,71.495219,73.547575,72.526247,70.406198,66.553949,66.577803,68.881273,76.100961,...,3.167156,2.892389,2.649189,2.608579,2.758104,2.694691,2.807125,2.811854,2.980309,3.229625


In [26]:
meteoM.to_parquet('meteoM.gzip',compression='gzip', engine='pyarrow')

In [27]:
%%time
meteoM = pd.read_parquet('meteoM.gzip', engine='pyarrow')
meteoM.dtypes

Wall time: 14 ms


01Humidite       float64
02Humidite       float64
03Humidite       float64
04Humidite       float64
05Humidite       float64
                  ...   
08VitesseVent    float64
09VitesseVent    float64
10VitesseVent    float64
11VitesseVent    float64
12VitesseVent    float64
Length: 72, dtype: object

In [28]:
meteoM.head()

,,,,,01Humidite,02Humidite,03Humidite,04Humidite,05Humidite,06Humidite,07Humidite,08Humidite,09Humidite,10Humidite,...,03VitesseVent,04VitesseVent,05VitesseVent,06VitesseVent,07VitesseVent,08VitesseVent,09VitesseVent,10VitesseVent,11VitesseVent,12VitesseVent
Nom,Latitude,Longitude,Altitude,Zone,,,,,,,,,,,,,,,,,,,,,
Abbeville,50.136000,1.834000,69,NO,86.724713,83.442240,81.179400,75.727027,78.056258,79.027077,77.900293,78.544567,80.426669,84.614636,...,4.825237,4.297918,4.199157,3.891925,3.913016,3.722128,3.732194,4.218847,4.305830,4.752122
Ajaccio,41.918000,8.792667,5,SE,77.577222,76.085307,74.541455,74.441117,73.218376,70.520309,67.527176,67.632660,70.216669,74.434473,...,3.278142,3.291029,3.310744,3.441544,3.601828,3.502506,3.381423,3.091826,3.145965,3.233035
Alencon,48.445500,0.110167,143,NO,88.713347,84.135055,79.976677,74.493544,75.708525,74.197207,71.743615,73.341054,76.939384,84.826907,...,3.676683,3.435019,3.412068,3.196204,3.258630,2.959461,2.865299,3.087170,3.266048,3.620345
Bale,47.614333,7.510000,263,NE,84.468035,79.617506,73.838076,69.405712,71.928629,71.442922,71.057795,74.507434,79.220094,83.978486,...,2.889943,2.704043,2.699644,2.451203,2.394586,2.131648,2.148799,2.313112,2.467429,2.889509
Bastia,42.540667,9.485167,10,SE,73.027388,71.772398,71.495219,73.547575,72.526247,70.406198,66.553949,66.577803,68.881273,76.100961,...,3.167156,2.892389,2.649189,2.608579,2.758104,2.694691,2.807125,2.811854,2.980309,3.229625


# La météo 	hebdomadaire

In [29]:
meteoH = meteo[['Nom', 'Latitude', 'Longitude', 'Altitude','Zone', 
                'VitesseVent', 'Temperature', 'Humidite', 
                'Visibilite','Pression','Precipitation', 
                'Semaine']].pivot_table(index=['Nom', 'Latitude', 'Longitude', 'Altitude','Zone'],
                           columns='Semaine')
meteoH.columns = [f'{x[1]:02d}{x[0]}' for x in meteoH.columns]
meteoH.head()

,,,,,01Humidite,02Humidite,03Humidite,04Humidite,05Humidite,06Humidite,07Humidite,08Humidite,09Humidite,10Humidite,...,44VitesseVent,45VitesseVent,46VitesseVent,47VitesseVent,48VitesseVent,49VitesseVent,50VitesseVent,51VitesseVent,52VitesseVent,53VitesseVent
Nom,Latitude,Longitude,Altitude,Zone,,,,,,,,,,,,,,,,,,,,,
Abbeville,50.136000,1.834000,69,NO,86.688060,87.020896,86.264531,86.573034,86.007468,82.972367,84.353338,82.897837,81.947839,82.798200,...,4.237357,4.578168,4.017980,4.495269,4.436497,4.747341,4.695489,4.629707,4.937481,4.716071
Ajaccio,41.918000,8.792667,5,SE,77.585714,79.056929,78.520864,75.343563,77.171642,75.588367,76.082774,76.297096,75.520864,74.777032,...,3.150395,3.254277,3.087167,3.169468,3.097783,3.358982,3.255821,3.004286,3.302992,3.530714
Alencon,48.445500,0.110167,143,NO,89.482476,89.214605,89.207154,87.235821,86.826607,84.339312,84.767267,83.362819,81.783784,81.551081,...,3.172981,3.483684,3.088326,3.227945,3.409993,3.770874,3.559715,3.372380,3.819925,3.497101
Bale,47.614333,7.510000,263,NE,85.271780,85.777198,83.521966,83.501493,82.726935,80.749441,80.520477,77.888473,76.306031,74.143708,...,2.419242,2.611676,2.288164,2.539357,2.543257,2.931315,3.073040,2.759552,2.947946,2.729032
Bastia,42.540667,9.485167,10,SE,72.947644,74.131184,73.303892,72.084021,72.813483,70.550150,70.501864,73.839432,71.792382,71.106175,...,2.868988,2.806102,2.938423,3.334820,2.911367,3.315281,3.274286,3.187897,3.215258,3.001071


In [30]:
meteoH.isna().sum()[meteoH.isna().sum() > 0]

Series([], dtype: int64)

In [31]:
meteoH.to_parquet('meteoH.gzip',compression='gzip', engine='pyarrow')

In [32]:
%%time
meteoH = pd.read_parquet('meteoH.gzip', engine='pyarrow')
meteoH.dtypes

Wall time: 37.9 ms


01Humidite       float64
02Humidite       float64
03Humidite       float64
04Humidite       float64
05Humidite       float64
                  ...   
49VitesseVent    float64
50VitesseVent    float64
51VitesseVent    float64
52VitesseVent    float64
53VitesseVent    float64
Length: 318, dtype: object

In [33]:
meteoH.head()

,,,,,01Humidite,02Humidite,03Humidite,04Humidite,05Humidite,06Humidite,07Humidite,08Humidite,09Humidite,10Humidite,...,44VitesseVent,45VitesseVent,46VitesseVent,47VitesseVent,48VitesseVent,49VitesseVent,50VitesseVent,51VitesseVent,52VitesseVent,53VitesseVent
Nom,Latitude,Longitude,Altitude,Zone,,,,,,,,,,,,,,,,,,,,,
Abbeville,50.136000,1.834000,69,NO,86.688060,87.020896,86.264531,86.573034,86.007468,82.972367,84.353338,82.897837,81.947839,82.798200,...,4.237357,4.578168,4.017980,4.495269,4.436497,4.747341,4.695489,4.629707,4.937481,4.716071
Ajaccio,41.918000,8.792667,5,SE,77.585714,79.056929,78.520864,75.343563,77.171642,75.588367,76.082774,76.297096,75.520864,74.777032,...,3.150395,3.254277,3.087167,3.169468,3.097783,3.358982,3.255821,3.004286,3.302992,3.530714
Alencon,48.445500,0.110167,143,NO,89.482476,89.214605,89.207154,87.235821,86.826607,84.339312,84.767267,83.362819,81.783784,81.551081,...,3.172981,3.483684,3.088326,3.227945,3.409993,3.770874,3.559715,3.372380,3.819925,3.497101
Bale,47.614333,7.510000,263,NE,85.271780,85.777198,83.521966,83.501493,82.726935,80.749441,80.520477,77.888473,76.306031,74.143708,...,2.419242,2.611676,2.288164,2.539357,2.543257,2.931315,3.073040,2.759552,2.947946,2.729032
Bastia,42.540667,9.485167,10,SE,72.947644,74.131184,73.303892,72.084021,72.813483,70.550150,70.501864,73.839432,71.792382,71.106175,...,2.868988,2.806102,2.938423,3.334820,2.911367,3.315281,3.274286,3.187897,3.215258,3.001071


# La météo 	journalière

In [34]:
meteoJ = meteo[['Nom', 'Latitude', 'Longitude', 'Altitude','Zone', 
                'VitesseVent', 'Temperature', 'Humidite', 
                'Visibilite','Pression','Precipitation', 
                'MoisJour']].pivot_table(index=['Nom', 'Latitude', 'Longitude', 'Altitude','Zone'],
                           columns='MoisJour')
meteoJ.columns = [f'{x[1]:04d}{x[0]}' for x in meteoJ.columns]
meteoJ.head()

,,,,,0101Humidite,0102Humidite,0103Humidite,0104Humidite,0105Humidite,0106Humidite,0107Humidite,0108Humidite,0109Humidite,0110Humidite,...,1222VitesseVent,1223VitesseVent,1224VitesseVent,1225VitesseVent,1226VitesseVent,1227VitesseVent,1228VitesseVent,1229VitesseVent,1230VitesseVent,1231VitesseVent
Nom,Latitude,Longitude,Altitude,Zone,,,,,,,,,,,,,,,,,,,,,
Abbeville,50.136000,1.834000,69,NO,88.401042,86.432292,86.613757,85.442708,87.031250,87.088542,85.843750,86.229167,87.307292,88.067708,...,5.267708,5.104167,5.341146,5.144271,5.063542,5.132632,4.656771,4.508333,4.784896,4.217801
Ajaccio,41.918000,8.792667,5,SE,76.786458,78.052356,77.905263,79.223404,78.432292,77.368421,76.677419,78.946809,79.052356,80.078534,...,2.962500,3.067914,2.898438,3.303646,3.093194,3.451042,4.306283,3.204688,2.863351,3.222396
Alencon,48.445500,0.110167,143,NO,90.389474,87.801047,88.656250,89.416667,89.947917,90.500000,89.411458,89.322917,88.411458,89.526042,...,3.813542,3.725926,4.006842,4.176440,3.853646,4.142328,3.494792,3.186458,3.390576,3.359162
Bale,47.614333,7.510000,263,NE,87.229167,85.546875,83.541667,84.833333,83.989583,86.442708,85.937173,86.161458,87.178010,87.088542,...,3.105208,2.338743,2.733508,3.115104,3.208901,3.217368,3.505729,2.923037,2.705208,2.365969
Bastia,42.540667,9.485167,10,SE,73.890625,72.806283,67.423280,69.416667,70.286458,74.078947,76.872340,79.536458,77.095745,75.905759,...,3.446354,3.150262,2.836316,3.064171,3.414894,3.238220,3.752604,3.014583,3.011458,3.117277


In [35]:
meteoJ.to_parquet('meteoJ.gzip',compression='gzip', engine='pyarrow')

In [36]:
%%time
meteoJ = pd.read_parquet('meteoJ.gzip', engine='pyarrow')
meteoJ.dtypes

Wall time: 274 ms


0101Humidite       float64
0102Humidite       float64
0103Humidite       float64
0104Humidite       float64
0105Humidite       float64
                    ...   
1227VitesseVent    float64
1228VitesseVent    float64
1229VitesseVent    float64
1230VitesseVent    float64
1231VitesseVent    float64
Length: 2196, dtype: object

In [37]:
meteoJ.head()

,,,,,0101Humidite,0102Humidite,0103Humidite,0104Humidite,0105Humidite,0106Humidite,0107Humidite,0108Humidite,0109Humidite,0110Humidite,...,1222VitesseVent,1223VitesseVent,1224VitesseVent,1225VitesseVent,1226VitesseVent,1227VitesseVent,1228VitesseVent,1229VitesseVent,1230VitesseVent,1231VitesseVent
Nom,Latitude,Longitude,Altitude,Zone,,,,,,,,,,,,,,,,,,,,,
Abbeville,50.136000,1.834000,69,NO,88.401042,86.432292,86.613757,85.442708,87.031250,87.088542,85.843750,86.229167,87.307292,88.067708,...,5.267708,5.104167,5.341146,5.144271,5.063542,5.132632,4.656771,4.508333,4.784896,4.217801
Ajaccio,41.918000,8.792667,5,SE,76.786458,78.052356,77.905263,79.223404,78.432292,77.368421,76.677419,78.946809,79.052356,80.078534,...,2.962500,3.067914,2.898438,3.303646,3.093194,3.451042,4.306283,3.204688,2.863351,3.222396
Alencon,48.445500,0.110167,143,NO,90.389474,87.801047,88.656250,89.416667,89.947917,90.500000,89.411458,89.322917,88.411458,89.526042,...,3.813542,3.725926,4.006842,4.176440,3.853646,4.142328,3.494792,3.186458,3.390576,3.359162
Bale,47.614333,7.510000,263,NE,87.229167,85.546875,83.541667,84.833333,83.989583,86.442708,85.937173,86.161458,87.178010,87.088542,...,3.105208,2.338743,2.733508,3.115104,3.208901,3.217368,3.505729,2.923037,2.705208,2.365969
Bastia,42.540667,9.485167,10,SE,73.890625,72.806283,67.423280,69.416667,70.286458,74.078947,76.872340,79.536458,77.095745,75.905759,...,3.446354,3.150262,2.836316,3.064171,3.414894,3.238220,3.752604,3.014583,3.011458,3.117277


In [38]:
meteoJ.isna().sum()[meteoJ.isna().sum() > 0]

Series([], dtype: int64)